In [3]:
import tensorflow as tf
import numpy as np

In [67]:
class modified_gru():
    """
        It's a modified version of normal GRU
    """
    def __init__(self,time_steps,init_state,input_seq,scalar_values,input_size,
                 output_size,final_mem,question,init_word):
        
        # inputs required for sequential memory vectors e_i 
        self.time_steps = time_steps              # equal to max no concepts
        self.init_state = init_state              # initial secondary input
        self.input_seq = input_seq                # input seq to be evaluated
        self.scalar_values = scalar_values        # gated scalar values 
        self.input_size = input_size              # size of input vector 
        self.output_size = output_size            # size of output required
        
        # inputs required for Answer modules
        self.final_mem = final_mem
        self.question = question
        self.init_word = init_word          # should be ideally a zero vector
        
        # parameters of gru
        self.W_z,self.W_r,self.W_h = tf.get_variable("update_w",[input_size,output_size],dtype=tf.float64),tf.get_variable("reset_w",[output_size,input_size],dtype=tf.float64),tf.get_variable("out_w",[input_size,output_size],dtype=tf.float64)
        self.U_z,self.U_r,self.U_h = tf.get_variable("update_u",[input_size,output_size],dtype=tf.float64),tf.get_variable("reset_u",[input_size,output_size],dtype=tf.float64),tf.get_variable("out_u",[output_size,output_size],dtype=tf.float64)
        self.b_z,self.b_r,self.b_h = tf.get_variable("update_b",[output_size,1],dtype=tf.float64),tf.get_variable("reset_b",[output_size,1],dtype=tf.float64),tf.get_variable("out_b",[output_size,1],dtype=tf.float64)
    """
    right now its a normal gru !!!
    You need to change the the output !!!
    """
    def mod_gru_unit(self,h_prev,steps,out=[]):
        index = steps - self.time_steps 
        input_vector = self.input_seq[index]
        # operations
        z_t = tf.nn.sigmoid(tf.matmul(self.W_z,input_vector,name = 'matmul_1')+tf.matmul(self.U_z,h_prev)+self.b_z,name="matmul_2")
        r_t = tf.nn.sigmoid(tf.matmul(self.W_r,input_vector)+tf.matmul(self.U_r,h_prev)+self.b_r)
        h_t = tf.multiply(z_t,h_prev)+tf.multiply(1.-z_t,tf.nn.sigmoid(tf.matmul(self.W_h,input_vector)+tf.matmul(self.U_h,tf.multiply(r_t,h_prev))+self.b_h))
        h_t = (self.scalar_values[index]*h_t) + ((1-self.scalar_values[index])*h_prev)
        out.append(h_t)
        self.time_steps -= 1
        if self.time_steps == 0:
            return(out)
        else:
            return self.mod_gru_unit(h_prev = h_t,out = out,steps = steps) 
        
    def answer_gru_unit(self,h_prev,word_prev,steps,out=[]):
        index = steps - self.time_steps 
        print(self.question)
        print(word_prev)
        input_vector = tf.concat([word_prev,self.question],axis=0)
        # operations
        z_t = tf.nn.sigmoid(tf.matmul(self.W_z,input_vector)+tf.matmul(self.U_z,h_prev)+self.b_z)
        r_t = tf.nn.sigmoid(tf.matmul(self.W_r,input_vector)+tf.matmul(self.U_r,h_prev)+self.b_r)
        h_t = tf.multiply(z_t,h_prev)+tf.multiply(1.-z_t,tf.nn.sigmoid(tf.matmul(self.W_h,input_vector)+tf.matmul(self.U_h,tf.mul(r_t,h_prev))+self.b_h))
        word_pred = tf.nn.softmax(tf.matmul(self.word_weight,h_t)+self.word_bias)
        out.append(word_pred)
        self.time_steps -= 1
        if self.time_steps == 0:
            return(out)
        else:
            return self.answer_gru_unit(h_prev = h_t,word_prev = word_pred,out = out,steps = steps) 
    
       

In [68]:
tf.reset_default_graph()
gru = modified_gru(time_steps = 10,
                   init_state = np.random.random(size=(20,1)),
                   input_seq = np.random.random(size=(10,40,1)),
                   scalar_values = np.random.random(size=(10,1)),
                   input_size = 40,
                   output_size = 20,
                   final_mem = np.random.random(size=(20,1)),
                   question = np.random.random(size=(20,1)),
                   init_word = np.random.random(size=(20,1)))
gru.answer_gru_unit(h_prev=np.random.random(size=(20,1)),
                    word_prev = np.random.random(size=(20,1)),
                    steps = gru.time_steps)

[[0.94456412]
 [0.24608355]
 [0.01658627]
 [0.43262796]
 [0.74242072]
 [0.0461399 ]
 [0.31687924]
 [0.68313563]
 [0.08155249]
 [0.87750023]
 [0.96012676]
 [0.33325693]
 [0.41536547]
 [0.42453094]
 [0.99107523]
 [0.86855346]
 [0.48868001]
 [0.65963245]
 [0.13239421]
 [0.32442833]]
[[7.41165168e-03]
 [6.78715623e-01]
 [5.79413990e-01]
 [4.48973451e-01]
 [1.81159469e-01]
 [9.87749689e-01]
 [8.05120852e-01]
 [9.53027492e-01]
 [5.03207070e-01]
 [2.99581108e-01]
 [9.99215042e-01]
 [8.38380737e-01]
 [2.05319080e-04]
 [7.62210784e-01]
 [7.07516249e-01]
 [3.42378449e-01]
 [3.56734480e-01]
 [3.08184681e-02]
 [1.31654069e-01]
 [8.35943592e-01]]


ValueError: Dimensions must be equal, but are 20 and 40 for 'MatMul' (op: 'MatMul') with input shapes: [20,20], [40,1].